<a href="https://colab.research.google.com/github/lokeaichirou/LLM/blob/main/workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install Agently==3.3.4.5
!pip install mermaid-python
!pip installnest_asyncio

ERROR: unknown command "installnest_asyncio"


In [2]:
# 使用nest_asyncio确保异步稳定性
import nest_asyncio
nest_asyncio.apply()

In [3]:
deep_seek_url           = "https://api.deepseek.com"
deep_seek_api_key       = "sk-104cf02ecfbf41dbac6736e2bda05d26"
deep_seek_default_model = 'deepseek-chat'

In [4]:
import json
import Agently
import os

[WARNING][2024-10-09 11:17:51.925] redis_rate_limiter.py:21 [t:138239985475584]: no redis installed, RedisRateLimiter unavailable


In [5]:
# 创建一个predictor agent

analyst_1 = (Agently.create_agent()
                 .set_settings("current_model", "OAIClient")
                 .set_settings("model.OAIClient.url", deep_seek_url)
                 .set_settings("model.OAIClient.auth", {"api_key": deep_seek_api_key})
                 .set_settings("model.OAIClient.options", {"model": deep_seek_default_model })
             )

analyst_2 = (Agently.create_agent()
                 .set_settings("current_model", "OAIClient")
                 .set_settings("model.OAIClient.url", deep_seek_url)
                 .set_settings("model.OAIClient.auth", {"api_key": deep_seek_api_key})
                 .set_settings("model.OAIClient.options", {"model": deep_seek_default_model })
             )


# 创建两个工作流：主工作流和分块创作工作流
workflow = Agently.Workflow()

In [6]:
# 定义主工作流的工作块

## 输入一句话描述
@workflow.chunk()
def input_match_basic_info(inputs, storage):

    storage.set("refree_name", input("[💡请输入主裁判名字]:"))

    storage.set("competition_name", input("[💡请输入赛事名称]:"))
    storage.set("home_team", input("[💡请输入主场球队]:"))
    storage.set("away_team", input("[💡请输入客场球队]:"))
    storage.set("weather_condition", input("[💡请输入天气情况]:"))

    storage.set("home_team_coach", input("[💡请输入主场球队教练]:"))
    storage.set("away_team_coach", input("[💡请输入客场球队教练]:"))

    storage.set("home_team_line_up", input("[💡请输入主场球队首发11人名单]:"))
    storage.set("home_team_substitutions", input("[💡请输入主场球队替补队员名单]:"))
    storage.set("home_team_formation", input("[💡请输入主场球队阵型]:"))

    storage.set("away_team_line_up", input("[💡请输入客场球队首发11人名单]:"))
    storage.set("away_team_substitutions", input("[💡请输入客场球队替补队员名单]:"))
    storage.set("away_team_formation", input("[💡请输入客场球队阵型]:"))

    storage.set("home_team_tactics_style", input("[💡请输入主场球队战术风格]:"))
    storage.set("away_team_tactics_style", input("[💡请输入客场球队战术风格]:"))

    storage.set("home_team_recent_results", input("[💡请输入主场球队近期战绩]:"))
    storage.set("away_team_recent_results", input("[💡请输入客场球队近期战绩]:"))

    storage.set("home_team_player_current_season_stats", input("[💡请输入主场球队球员本赛季数据]:"))
    storage.set("away_team_player_current_season_stats", input("[💡请输入客场球队球员本赛季数据]:"))

    return

In [9]:
## 过程产出输出
@workflow.chunk_class()
def print_process_output(inputs, storage):

    print(f"[{ inputs['default']['title'] }]:")
    if isinstance(inputs["default"]["result"], dict):
        print(json.dumps(inputs["default"]["result"], indent=4, ensure_ascii=False))
    else:
        print(inputs["default"]["result"])
    return

In [10]:
## 创建比赛预览
@workflow.chunk()
def create_game_scenario(inputs, storage):

    confirm = "n"
    while confirm.lower() == "n":

        game_scenario       = storage.get("game_scenario", None)
        revision_suggestion = storage.get("revision_suggestion_for_game_scenario", None)

        if game_scenario and revision_suggestion:

            updated_game_scenario = (analyst_1.set_role({"角色", "你是一个足球分析评论员, 预估球队整体表现",})
                                              .input({"比赛预览": game_scenario, "比赛基本情况": game_scenario["比赛基本情况"], "两队各自优势与潜在问题": game_scenario["两队各自优势与潜在问题"],
                                                      "两队主帅战术安排": game_scenario["两队主帅战术安排"], "修改建议": revision_suggestion})
                                              .instruct("""请根据{修改建议}改进{比赛基本情况}、{两队各自优势与潜在问题}、{两队主帅战术安排}并更新至{比赛预览}里的相关条目""")
                                              .output({"比赛基本情况": [{"比赛风格":   ("str", "基于双方球队和主帅风格特点给出的比赛大体风格"),
                                                                       "比赛局面":   ("str", "基于双方球队和主帅风格特点给出的比赛大体局面"),
                                                                       "比赛激烈度": ("str", "基于双方球队和主帅风格特点给出的比赛激烈程度"),}
                                                                      ],

                                                       "两队各自优势与潜在问题": [{"主队在进攻端的优势": ("str", "给出主队在进攻端的优势"),
                                                                               "主队在中场的优势":   ("str", "给出主队在中场的优势"),
                                                                               "主队在防守端的优势": ("str", "给出主队在防守端的优势"),
                                                                               "客队在进攻端的优势": ("str", "给出客队在进攻端的优势"),
                                                                               "客队在中场的优势":   ("str", "给出客队在中场的优势"),
                                                                               "客队在防守端的优势": ("str", "给出客队在防守端的优势"),
                                                                               "主队在进攻端的潜在问题与隐患": ("str", "给出主队在进攻端的潜在问题与隐患"),
                                                                               "主队在中场的潜在问题与隐患":   ("str", "给出主队在中场的潜在问题与隐患"),
                                                                               "主队在防守端的潜在问题与隐患": ("str", "给出主队在防守端的潜在问题与隐患"),
                                                                               "客队在进攻端的潜在问题与隐患": ("str", "给出客队在进攻端的潜在问题与隐患"),
                                                                               "客队在中场的潜在问题与隐患":   ("str", "给出客队在中场的潜在问题与隐患"),
                                                                               "客队在防守端的潜在问题与隐患": ("str", "给出客队在防守端的潜在问题与隐患"),}
                                                                               ],
                                                       "两队主帅战术安排": [{"主队主帅的战术安排": ("str",  "给出主队主帅的战术安排"),
                                                                          "客队主帅的战术安排": ("str", "给出客队主帅的战术安排"),}
                                                                         ]
                                                       }
                                                      ).start()
                                      )

            storage.set("game_scenario", updated_game_scenario)
            print(json.dumps(updated_game_scenario, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "比赛预览", "result": updated_game_scenario,}
            else:
                storage.set("revision_suggestion_for_game_scenario", input("[请输入您的修改意见]: "))

        else:

            competition_name = storage.get("competition_name")
            home_team        = storage.get("home_team")
            away_team        = storage.get("away_team")

            home_team_coach = storage.get("home_team_coach")
            away_team_coach = storage.get("away_team_coach")

            home_team_line_up = storage.get("home_team_line_up")
            away_team_line_up = storage.get("away_team_line_up")

            home_team_substitutions = storage.get("home_team_substitutions")
            away_team_substitutions = storage.get("away_team_substitutions")

            home_team_formation = storage.get("home_team_formation")
            away_team_formation = storage.get("away_team_formation")

            home_team_tactics_style = storage.get("home_team_tactics_style")
            away_team_tactics_style = storage.get("away_team_tactics_style")

            game_scenario = (analyst_1.set_role({"角色", "你是一个足球分析评论员, 预估球队整体表现",})
                                      .input({"赛事名称": competition_name, "主场球队": home_team, "客场球队":away_team, "主队首发": home_team_line_up, "客队首发":away_team_line_up,
                                              "主队替补": home_team_substitutions, "客队替补": away_team_substitutions, "主队阵型": home_team_formation, "客队阵型":away_team_formation,
                                              "主队主帅": home_team_coach, "客队主帅": away_team_coach, "主队技战术风格": home_team_tactics_style, "客队技战术风格": away_team_tactics_style})
                                      .instruct("""请根据{赛事名称}和驻主场因素，并基于你对{主场球队}和{客场球队}的当前各自实力、过往技战术风格和过往恩怨的了解，结合{主队主帅}、{客队主帅}的执教风格以及本场比赛{主队技战术风格}、{客队技战术风格}，分析评估一下比赛大体风格、局面和激烈程度
                                                    同时基于你对{主队首发}、{主队替补}、{客队首发}、{客队替补}的了解和基于{主队阵型}与{客队阵型}产生的对位情况分析预估一下比赛的进一步具体情况, 体现在两队在各自攻防两端和中场的优势与潜在问题与隐患
                                                    结合上面的分析以及{主队主帅}、{客队主帅}他们的执教特点与换人习惯评估下可能的战术安排""")
                                      .output({"比赛基本情况": [{"比赛风格":   ("str", "基于双方球队和主帅风格特点给出的比赛大体风格"),
                                                               "比赛局面":   ("str", "基于双方球队和主帅风格特点给出的比赛大体局面"),
                                                               "比赛激烈度": ("str", "基于双方球队和主帅风格特点给出的比赛激烈程度"),}
                                                              ],

                                              "两队各自优势与潜在问题": [{"主队在进攻端的优势": ("str", "给出主队在进攻端的优势"),
                                                                      "主队在中场的优势":   ("str", "给出主队在中场的优势"),
                                                                      "主队在防守端的优势": ("str", "给出主队在防守端的优势"),
                                                                      "客队在进攻端的优势": ("str", "给出客队在进攻端的优势"),
                                                                      "客队在中场的优势":   ("str", "给出客队在中场的优势"),
                                                                      "客队在防守端的优势": ("str", "给出客队在防守端的优势"),
                                                                      "主队在进攻端的潜在问题与隐患": ("str", "给出主队在进攻端的潜在问题与隐患"),
                                                                      "主队在中场的潜在问题与隐患":   ("str", "给出主队在中场的潜在问题与隐患"),
                                                                      "主队在防守端的潜在问题与隐患": ("str", "给出主队在防守端的潜在问题与隐患"),
                                                                      "客队在进攻端的潜在问题与隐患": ("str", "给出客队在进攻端的潜在问题与隐患"),
                                                                      "客队在中场的潜在问题与隐患":   ("str", "给出客队在中场的潜在问题与隐患"),
                                                                      "客队在防守端的潜在问题与隐患": ("str", "给出客队在防守端的潜在问题与隐患"),}
                                                                      ],
                                              "两队主帅战术安排": [{"主队主帅的战术安排": ("str",  "给出主队主帅的战术安排"),
                                                                 "客队主帅的战术安排": ("str", "给出客队主帅的战术安排"),}
                                                                ]
                                              }
                                             ).start()
                              )

            storage.set("game_scenario", game_scenario)
            print(json.dumps(game_scenario, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "比赛预览", "result": game_scenario,}
            else:
                storage.set("revision_suggestion_for_game_scenario", input("[请输入您的修改意见]: "))


In [14]:
## 创建首发球员表现预估

@workflow.chunk()
def create_player_performance(inputs, storage):

    confirm = "n"
    while confirm.lower() == "n":

        line_up_players_performance = storage.get("line_up_players_performance", None)
        revision_suggestion         = storage.get("revision_suggestion_for_player_performance", None)

        if line_up_players_performance and revision_suggestion:

            updated_line_up_players_performance = (analyst_2.set_role({"角色", "你是一个足球数据分析师, 预估球员表现",})
                                                            .input({"首发球员表现预估": line_up_players_performance,
                                                                    "主队首发球员表现": line_up_players_performance["主队首发球员表现"],
                                                                    "客队首发球员表现": line_up_players_performance["客队首发球员表现"],
                                                                    "修改建议": revision_suggestion
                                                                      }
                                                                    )
                                                            .instruct("""请根据{修改建议}改进{主队首发球员表现}、{客队首发球员表现}并更新至{首发球员表现预估}里的相关条目""")
                                                            .output({"主队首发球员表现": [{"球员姓名": ("str", ), "大致表现及状态": ("str", ),}],
                                                                     "客队首发球员表现": [{"球员姓名": ("str", ), "大致表现及状态": ("str", ),}],
                                                                      }
                                                                    )
                                                            .start()
                                                    )

            storage.set("line_up_players_performance", updated_line_up_players_performance)
            print(json.dumps(updated_line_up_players_performance, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "首发球员表现预估", "result": updated_line_up_players_performance,}
            else:
                storage.set("revision_suggestion_for_player_performance", input("[请输入您的修改意见]: "))

        else:
            game_scenario      = storage.get("game_scenario")

            home_team_line_up  = storage.get("home_team_line_up")
            away_team_line_up  = storage.get("away_team_line_up")

            home_team_player_current_season_stats = storage.get("home_team_player_current_season_stats")
            away_team_player_current_season_stats = storage.get("away_team_player_current_season_stats")

            line_up_players_performance = (analyst_2.set_role({"角色", "你是一个足球数据分析师, 预估球员表现",})
                                                    .input({"比赛基本情况": game_scenario["比赛基本情况"], "两队各自优势与潜在问题": game_scenario["两队各自优势与潜在问题"], "两队主帅战术安排": game_scenario["两队主帅战术安排"],
                                                            "主队首发": home_team_line_up, "客队首发": away_team_line_up, "主场球队球员本赛季数据": home_team_player_current_season_stats, "客场球队球员本赛季数据": away_team_player_current_season_stats})
                                                    .instruct("""请根据{比赛基本情况}、{两队各自优势与潜在问题}和{两队主帅战术安排}, 结合你对{主队首发}、{客队首发}球员的了解以及{主场球队球员本赛季数据}、{客场球队球员本赛季数据}分析预估下场上首发球员的预期表现和状态""")
                                                    .output({"主队首发球员表现": [{"球员姓名": ("str", ), "大致表现及状态": ("str", ),}],
                                                             "客队首发球员表现": [{"球员姓名": ("str", ), "大致表现及状态": ("str", ),}],
                                                              }
                                                            ).start()
                                            )

            storage.set("line_up_players_performance", line_up_players_performance)
            print(json.dumps(line_up_players_performance, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "首发球员表现预估", "result": line_up_players_performance,}
            else:
                storage.set("revision_suggestion_for_player_performance", input("[请输入您的修改意见]: "))

In [15]:
(workflow.chunks["START"]
 .connect_to("input_match_basic_info")
 .connect_to("create_game_scenario")
 .connect_to("@print_process_output")
 .connect_to("create_player_performance")
 .connect_to("@print_process_output")
 .connect_to("END")
 )

In [16]:
workflow.draw(type="img")

In [17]:
# 开始执行
result = workflow.start()
print(result["default"])

[💡请输入主裁判名字]:大卫奥利弗
[💡请输入赛事名称]:英格兰足球超级联赛
[💡请输入主场球队]:阿森纳
[💡请输入客场球队]:利物浦
[💡请输入天气情况]:晴天
[💡请输入主场球队教练]:阿尔特塔
[💡请输入客场球队教练]:阿尔恩-斯洛特
[💡请输入主场球队首发11人名单]:大卫拉亚、卡拉菲奥里、加布里埃尔、萨利巴、廷贝尔、赖斯、托马斯帕泰、厄德高、马丁内利、哈弗茨、萨卡
[💡请输入主场球队替补队员名单]:拉希姆斯特林，热苏斯、富安健洋、蒂尔尼、若日尼奥、基维奥尔、恩瓦内里，米克尔梅里诺，津琴科
[💡请输入主场球队阵型]:4-3-3
[💡请输入客场球队首发11人名单]:阿利森、罗伯逊、范戴克、科纳特、阿诺德、麦卡利斯特、赫拉芬贝赫、路易斯迪亚斯、索博斯洛伊、萨拉赫、达尔文努涅斯
[💡请输入客场球队替补队员名单]:琼斯，加克波，乔-戈麦斯，凯莱赫，科纳布拉德利，远藤航，宽萨，费德里科-基耶萨，埃利奥特
[💡请输入客场球队阵型]:4-2-3-1
[💡请输入主场球队战术风格]:实用性的地面进攻加控球和一定程度的高位压迫，注重角球与定位球在进攻中的作用，防守时注重身体对抗与强度
[💡请输入客场球队战术风格]:一定程度沿袭了克洛普时期的压迫打法，跑动积极，有强调中场洗球，防守时也倚赖范戴克为首的个人能力
[💡请输入主场球队近期战绩]:英超目前4胜2平0负，进12球失5球
[💡请输入客场球队近期战绩]:英超目前5胜0平1负，进12球失2球
[💡请输入主场球队球员本赛季数据]:热苏斯英超出场4次，其中首发0次替补4次，0球0助3抢断3犯规2黄牌。斯特林英超出场2次，其中首发0次替补2次，0球0助。哈弗茨英超出场6次，其中首发6次, 3球1助5抢断11犯规0黄牌。萨卡英超出场6次，其中首发6次，1球5助10抢断3犯规1黄牌。马丁内利英超出场6次，其中首发5次替补1次，1球2助5抢断4犯规0黄牌。恩瓦内里英超出场2次，其中首发0次替补2次，0球0助。托马斯帕泰英超出场6次，其中首发6次,1球0助17抢断10犯规2黄牌。赖斯英超出场5次，其中首发5次，0球0助2抢断4犯规2黄牌1红牌。梅里诺英超出场0次。若日尼奥英超出场1次，其中首发1次，0球0助1抢断2犯规1黄牌。厄德高英超出场3次，其中首发3次，0球0助3抢断1犯规1黄牌。津琴科英超出场2次，其中首发1次替补1次，0球0助0抢

In [ ]:
## 创建比赛情节线

@workflow.chunk()
def generate_game_line(inputs, storage):

